# Run.c

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karpathy/llama2.c/blob/master/run.ipynb)

More details can be found in the [README.md](README.md) .

In [11]:
#@title Clone Project
!rm -rf tem-force
!git clone https://github.com/yyj6666667/tem-force
!ls
%cd tem-force

Cloning into 'tem-force'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 36), reused 91 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 1.08 MiB | 4.36 MiB/s, done.
Resolving deltas: 100% (36/36), done.
assets		 model.py	    runq	     tinystories.py
build_msvc.bat	 nvprof_output.txt  runq.c	     tokenizer.bin
configurator.py  __pycache__	    run_version1.cu  tokenizer.model
doc		 README.md	    run_version2.cu  tokenizer.py
export.py	 requirements.txt   sample.py	     train.py
FlameGraph	 run		    stories42M.bin   win.c
improvement	 run.c		    tem-force	     win.h
LICENSE		 run.cu		    test_all.py      yyj_report.nsys-rep
Makefile	 run.ipynb	    test.c	     yyj_report.sqlite
/content/tem-force/tem-force


In [12]:
#@title Pick Your Model

#@markdown Choose model
model = "stories42M" #@param ["stories15M", "stories42M", "stories110M"]

download_url = ""

if(model == "stories15M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin"
if(model == "stories42M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin"
if(model == "stories110M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin"

print(f"download_url: {download_url}")

!wget $download_url

model_file = model + ".bin"

download_url: https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
--2026-01-10 14:56:52--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
Resolving huggingface.co (huggingface.co)... 3.168.132.40, 3.168.132.126, 3.168.132.91, ...
Connecting to huggingface.co (huggingface.co)|3.168.132.40|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/64c19eb0275dc8cd7f170b8f/20ab6952f8b77338484a45f788d50ed30dfda285f9954758b75d7deefd120d26?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27stories42M.bin%3B+filename%3D%22stories42M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1768060612&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MDYwNjEyfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjRjMTllYjAyNzVkYzhjZDdmMTcwYjhmLzIwYWI2OTUyZjhiNzczMzg0ODRhNDVmNzg4ZDUwZWQzMGRmZGEyODVmOTk1NDc1OGI3NWQ3ZGVlZm

In [42]:
#CPU;
!make runfast

gcc -Ofast -o run run.c -lm
gcc -Ofast -o runq runq.c -lm


若使用 CUDA 优化，需执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm命令编译代码；若使用了 cuBLAS 库，则执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm -L/usr/local/lib64 -lcublas编译代码；若采用 AVX、OpenMP 等其他优化方法，则直接执行make run即可。

In [13]:
#GPU
#!nvcc -O3 -o run run.cu -lcudart -lm

!nvcc -O3 -std=c++14 -arch=sm_75 -o run run.cu

run.cu(539): warning #2464-D: conversion from a string literal to "char *" is deprecated
          int dummy_prefix = str_lookup(" ", t->sorted_vocab, t->vocab_size);
                                        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

run.cu(786): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *empty_prompt = "";
                           ^

run.cu(966): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *tokenizer_path = "tokenizer.bin";
                             ^

run.cu(972): warning #2464-D: conversion from a string literal to "char *" is deprecated
      char *mode = "generate";
                   ^

run.cu(539): warning #2464-D: conversion from a string literal to "char *" is deprecated
          int dummy_prefix = str_lookup(" ", t->sorted_vocab, t->vocab_size);
                                        ^

Remark: The warnings can be suppressed wit

In [14]:

!apt-get install -y cuda-nsight-systems-12-2 >/dev/null || echo "nsys is installed"

!nvprof --version
!nsys --version

nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2024 NVIDIA Corporation
Release version 12.5.82 (21)
NVIDIA Nsight Systems version 2023.2.3.1004-33186433v0


In [15]:
#备选方案nvprof

# 2. 运行 nvprof
!nvprof --print-gpu-summary ./run stories42M.bin  2> nvprof_output.txt
# 步骤 1: 提取 GPU activities 部分
!cat nvprof_output.txt

Once upon a time, there was a little girl named Lily. She had a lot of toys, but she didn't have any more. So, she asked her mom to buy her some new toys. Her mom said, "Sure, let's go to the store and buy some."
When they got to the store, Lily saw a big stack of dolls. They were so pretty and expensive. She asked her mom if she could have them, but her mom said they were too expensive.
Lily was sad, but then she saw a small toy that she liked even more. It was a little teddy bear. She asked her mom if she could have it instead, and her mom said yes. Lily was very happy and hugged her new toy tightly. She thanked her mom and said, "I love my new toy!"
==4250== NVPROF is profiling process 4250, command: ./run stories42M.bin
achieved tok/s: 15.193824
==4250== Profiling application: ./run stories42M.bin
==4250== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   94.46%  5.23287s     21204  246.79us     767ns  29.149ms  [

In [16]:
#@title Generate Stories

# Generate args
max_token = 256 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
prompt = "One day, Lily met a Shoggoth" #@param {type:"string"}

print(f"model: {model_file}, max_token: {max_token}, temperature: {temperature}, top_p: {top_p}, prompt: {prompt}")
print(f"----------------------------\n")

cmd = f'./run {model_file} -t {temperature} -p {top_p} -n {max_token} -i "{prompt}"'
!{cmd}

model: stories42M.bin, max_token: 256, temperature: 0.8, top_p: 0.9, prompt: One day, Lily met a Shoggoth
----------------------------

One day, Lily met a Shoggoth. He was called Shoggy and he was very friendly. Lily liked him a lot. 
"Hi Shoggy," said Lily. "I'm so glad you're here. What's your name?"
Shoggy smiled and said, "My name is Shoggy. What's yours?"
Lily said, "My name is Lily. Do you want to play?"
Shoggy said, "Yes, I do. I'm so glad you like me. Let's play a game!"
Lily and Shoggy played together for a while. Then, Lily had to go home. She said goodbye to Shoggy and gave him a big hug.
"I'll miss you, Shoggy," said Lily. "Thank you for playing with me!"
Shoggy said, "I'll miss you too, Lily. Have a good day!"
achieved tok/s: 15.899329


In [17]:



# 详细采集（包含 CUDA、cuBLAS、cuDNN、NVTX、OS 运行时）
!nsys profile -t cuda,cublas,cudnn,nvtx,osrt --stats=true -o yyj_report ./run stories42M.bin


Once upon a time, there was a green frog named Freddy. Freddy lived in a small pond with his family. One day, Freddy saw a beautiful butterfly. He wanted to be friends with the butterfly.
Freddy said, "Hi, butterfly! I am Freddy. Do you want to play with me?" The butterfly said, "Yes, Freddy! I am Betty. Let's play a quiz game." They played the quiz game together and had a lot of fun.
But then, Freddy saw that Betty was sad. Freddy knew that Betty did not have any friends. So, he decided to forgive Betty for being sad. Freddy said, "Betty, it's okay. We all make mistakes. Let's be friends again." Betty smiled and said, "Thank you, Freddy. I am happy now." They played together every day and became best friends.
achieved tok/s: 14.879425
Generating '/tmp/nsys-report-258a.qdstrm'
[1/8] [========================100%] yyj_report.nsys-rep
[2/8] [========================100%] yyj_report.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/tem-force/tem-force/yyj_report.sqlite does